# Library: sp
Data: Philadelphia database. Source: .csv, .shp

In [ ]:
# Import libraries
library(sp)
library(rgdal)
library(rgeos)

In [ ]:
# Open files

# Open Philadelphia homicides database
path_hom <- "./data/PhillyHomicides/"
philly_hom <- readOGR(dsn=path_hom, "PhillyHomicides")

# Open Philadelphia education database
path_edu <- "./data/PhiladelphiaEduAttain.csv"
philly_edu <- read.csv(path_edu)

# Open Philadelphia region database
path <- "./data/Philly/"
philly <- readOGR(dsn=path, "PhillyTotalPopHHinc")

In [ ]:
# Display header
names(philly_edu)

In [ ]:
# Display info about datasets
str(philly)
str(philly_hom)

In [ ]:
# Median household income
spplot(philly, "medHHinc", main="Mediana dochodu na mieszkanie")

In [ ]:
# Subset where household income is more than 40 000$
plot(philly, main="Obszary z dochodem powyżej 40 000$")
philly_40k <- subset(philly, medHHinc > 40000)
plot(philly_40k, add=T, col="blue", main="Obszary z dochodem powyżej 40 000$")

In [ ]:
# Total population
spplot(philly, "totalPop")

In [ ]:
# Sampling
par(mfrow=c(2, 2))

regular <- spsample(philly, 100, type="regular", pretty=TRUE)
plot(regular)

random <- spsample(philly, 100, type="random", pretty=TRUE)
plot(random)

hexagonal <- spsample(philly, 100, type="hexagonal", pretty=TRUE)
plot(hexagonal)

stratified <- spsample(first_region_shape_area, 100, type="stratified")
plot(stratified)

In [ ]:
# Join database
philly_merge_edu <- merge(philly, philly_edu, by.x="GEOID10", by.y="GEOID")

# Dislay results
names(philly_merge_edu)

In [ ]:
# Reprojecting

# Check current coordinates
range(coordinates(philly))
range(coordinates(philly_hom))

proj4string(philly)
proj4string(philly_hom)

# Transform
philly_hom_proj <- spTransform(philly_hom, CRS(proj4string(philly)))

# Check results
range(coordinates(philly))
range(coordinates(philly_hom_proj))

proj4string(philly)
proj4string(philly_hom_proj)

In [ ]:
# Spatial aggregation
philly_aggr <- aggregate(x=philly_hom_proj["OBJ_ID"], by=philly, FUN=length)

In [ ]:
# Number of murders in Phipadelphia
spplot(philly[,1], colorkey=TRUE, sp.layout=philly_hom_proj["OBJ_ID"], main="Liczba morderstw w Filadelfii")
spplot(philly_aggr, main="Liczba morderstw w Filadelfii")
spplot(philly_hom["TEXT_GE"], main="Morderstwa w Filadelfii")